In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import confusion_matrix, multilabel_confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.multioutput import MultiOutputRegressor

# Load Datasets

In [8]:
def load_data(file_path):
    A = np.loadtxt(file_path)
    X = A[:, :9]    # Input features
    y = A[:, 9:]    # Output labels
    return X, y

In [9]:
# Load all 3 datasets
X_final, y_final = load_data('data/tictac_final.txt')
y_final = y_final[:, 0]

X_single, y_single = load_data('data/tictac_single.txt')
y_single = y_single[:, 0]

X_multi, y_multi = load_data('data/tictac_multi.txt')

## KNN Classifier

In [4]:
def knn_clf_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Define and train model
    model = KNeighborsClassifier(n_neighbors=1)
    model.fit(X_train, y_train)

    # Get cross validation accuracy
    val_accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    val_accuracy = np.mean(val_accuracy)

    # Get test accuracy
    y_pred = model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_pred)

    # Get confusion matrix
    confusion_mtrx = confusion_matrix(y_test, y_pred, normalize="true")

    return val_accuracy, test_accuracy, confusion_mtrx

In [5]:
def print_clf_results(val_accuracy, test_accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of KNN Classification on {dataset_name}:")
    print("Cross Validation Accuracy = ", val_accuracy)
    print("Test Accuracy = ", test_accuracy)
    print("Confusion Matrix:")
    print(confusion_mtrx.round(decimals=3)) # Round to 3 decimal places

## Final Dataset

In [10]:
val_acc_final, test_acc_final, confusion_mtrx_final = knn_clf_train(X_final, y_final)
print_clf_results(val_acc_final, test_acc_final, confusion_mtrx_final, "Final Dataset")

Performance of KNN Classification on Final Dataset:
Cross Validation Accuracy =  1.0
Test Accuracy =  1.0
Confusion Matrix:
[[1. 0.]
 [0. 1.]]


## Single Dataset

In [15]:
val_acc_single, test_acc_single, confusion_mtrx_single = knn_clf_train(X_single, y_single)
print_clf_results(val_acc_single, test_acc_single, confusion_mtrx_single, "Single Dataset")

Performance of KNN Classification on Single Dataset:
Accuracy =  0.8269083969465649
Confusion Matrix:
[[0.898 0.003 0.028 0.    0.04  0.009 0.012 0.006 0.003]
 [0.024 0.857 0.018 0.018 0.036 0.    0.012 0.    0.036]
 [0.08  0.016 0.856 0.011 0.011 0.005 0.005 0.005 0.011]
 [0.043 0.068 0.034 0.769 0.026 0.    0.009 0.017 0.034]
 [0.054 0.04  0.02  0.    0.861 0.    0.01  0.    0.015]
 [0.053 0.039 0.039 0.    0.    0.855 0.    0.    0.013]
 [0.02  0.03  0.    0.    0.02  0.    0.909 0.01  0.01 ]
 [0.02  0.1   0.    0.06  0.06  0.    0.02  0.68  0.06 ]
 [0.022 0.056 0.011 0.011 0.011 0.    0.    0.022 0.865]]




# KNN Regressor

In [16]:
def knn_reg_train(X, y):
    # Split into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

    # Define and train model
    base_classifier = KNeighborsRegressor(n_neighbors=1)
    model = MultiOutputRegressor(base_classifier)
    model.fit(X_train, y_train)

    # Get accuracy
    accuracy = cross_val_score(model, X_train, y_train, cv=10, scoring="accuracy")
    mean_accuracy = np.mean(accuracy)

    # Get confusion matrix
    y_pred = model.predict(X_test)
    confusion_mtrx = multilabel_confusion_matrix(y_test, y_pred)

    return mean_accuracy, confusion_mtrx

In [17]:
def print_reg_results(accuracy, confusion_mtrx, dataset_name):
    print(f"Performance of KNN Regression on {dataset_name}:")
    print("Accuracy = ", accuracy)
    print("Confusion Matrix:")

    for i, matrix in enumerate(confusion_mtrx):
        print(f"Confusion Matrix for Label {i}:")

        # Normalize confusion matrix
        row_sums = matrix.sum(axis=1, keepdims=True)
        matrix = matrix / row_sums
        print(matrix.round(decimals=3))
        
    print("\n")

## Multi Dataset

In [18]:
accuracy_multi, confusion_mtrx_multi = knn_reg_train(X_multi, y_multi)
print_reg_results(accuracy_multi, confusion_mtrx_multi, "Multi Dataset")

Performance of KNN Regression on Multi Dataset:
Accuracy =  0.7290076335877862
Confusion Matrix:
Confusion Matrix for Label 0:
[[0.955 0.045]
 [0.102 0.898]]
Confusion Matrix for Label 1:
[[0.954 0.046]
 [0.169 0.831]]
Confusion Matrix for Label 2:
[[0.975 0.025]
 [0.089 0.911]]
Confusion Matrix for Label 3:
[[0.964 0.036]
 [0.188 0.812]]
Confusion Matrix for Label 4:
[[0.956 0.044]
 [0.081 0.919]]
Confusion Matrix for Label 5:
[[0.961 0.039]
 [0.15  0.85 ]]
Confusion Matrix for Label 6:
[[0.958 0.042]
 [0.086 0.914]]
Confusion Matrix for Label 7:
[[0.949 0.051]
 [0.207 0.793]]
Confusion Matrix for Label 8:
[[0.951 0.049]
 [0.131 0.869]]


